In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#A function get url, direct ,the information on reservation time and df and returns an updated df with the 100 additional hotels.

def get_hotels(url , driver ,booking_df,Snapshot_date,checking_date_type,checkout_date_type):
    driver.get(url)
    hotel_name_list =[]
    page_number=1
    index = 1
    while(len(hotel_name_list) <=100 and page_number<=5): #take 100 hotel
        time.sleep(2)
        text_box = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
        for t_box in text_box:# run on full page
            if (len(hotel_name_list) >= 100):
                break
            try:
                hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Num of stars":None,"Grade":None,"Num of Reviews":None,"Curr Price":None,"Original Price":None,"Percentage of discount (Boolean)":None,"Distance from center":None,"Type of room":None,"Location grade":None}
                name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
                name = name.split('\n')[0]
                if(name in hotel_name_list):#Avoid hotels that are advertised several times
                    continue   
                hotel['Hotel Name'] = (name)
                hotel['Snapshot'] = Snapshot_date
                delta_TTT =  checking_date_type - Snapshot_date
                hotel['TTT'] = delta_TTT.days
                delta_LOS = checkout_date_type - checking_date_type
                hotel['LOS'] = delta_LOS.days
                score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
                score_detail_list = score_detail.split('\n')
                hotel["Grade"] = (score_detail_list[0])
                hotel["Num of Reviews"] = (score_detail_list[2])
                hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
                hotel["Index"] = index
                index = index +1
                booking_df = booking_df.append(hotel, ignore_index=True)
                hotel_name_list.append(name) 
            except:
                print("error")
        # change page 
        button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
        time.sleep(1.5)
        button.click() 
        page_number = page_number+1
    
    
    return booking_df
    

In [ ]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':0 }
driver = webdriver.Chrome(path['roy'])

In [ ]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)
        
print(combinations_list , len(combinations_list))        
    

In [ ]:
city = "new york"
numOfAdults=2
numOfChild=0
numOfRooms=1
booking_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Num of stars","Grade","Num of Reviews","Curr Price","Original Price","Percentage of discount (Boolean)","Distance from center","Type of room","Location grade"])  
for comb in combinations_list:
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
    booking_df = get_hotels(url, driver , booking_df, Snapshot_date,checking_date_type,checkout_date_type)
    

In [ ]:
booking_df.head(6500)

In [ ]:
booking_df.shape

In [ ]:
booking_df.groupby(['TTT','LOS']).count()

here we try find more thing

In [ ]:
path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':0 }
driver = webdriver.Chrome(path['roy'])


city = "new york"
checking_year = 2023
checking_month = 3
checking_monthday =1
checkout_year=2023
checkout_month=3
checkout_monthday=3
numOfAdults=2
numOfChild=0
numOfRooms=1
url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
driver.get(url)



In [ ]:
booking_df_test = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Num of stars","Grade","Num of Reviews","Curr Price","Original Price","Percentage of discount (Boolean)","Distance from center","Type of room","Location grade"])  

hotel_name_list =[]
page_number=1
index = 1
while(len(hotel_name_list) <100 and page_number<6): #take 100 hotel
    time.sleep(2)
    text_box = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
    for t_box in text_box:# run on full page
        try:
            hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Num of stars":None,"Grade":None,"Num of Reviews":None,"Curr Price":None,"Original Price":None,"Percentage of discount (Boolean)":None,"Distance from center":None,"Type of room":None,"Location grade":None}
            name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
            name = name.split('\n')[0]  
            hotel['Hotel Name'] = (name)
            score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
            score_detail_list = score_detail.split('\n')
            hotel["Grade"] = (score_detail_list[0])
            hotel["Num of Reviews"] = (score_detail_list[2])
            hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
            hotel["Index"] = index
            index = index +1
            booking_df_test.append(hotel, ignore_index=True)
            hotel_name_list.append(name) 
        except:
            print("error")
    # change page        
    button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
    time.sleep(1.5)
    button.click() 
    page_number = page_number+1
    
    

In [ ]:
booking_df_test.head(200)

In [ ]:
booking_df_test.shape